In [50]:
import xml.etree.ElementTree as ET
import csv

def process_tweets(xml_file, csv_file):
    """
    Procesa un archivo XML de tweets y crea un archivo CSV. Concatena todo el texto del tweet,
    incluso si hay múltiples etiquetas <sentiment>.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    header = ['tweet_text', 'aspects']

    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        

        for tweet in root.findall('tweet'):
            tweet_text = []
            for element in tweet.iter():
                if element.text:
                    tweet_text.append(element.text.strip()) 
                if element.tail and element.tail != '\n':
                    tweet_text.append(element.tail.strip()) 
            aspects = []

            for sentiment in tweet.findall('.//sentiment'):
                aspects.append(sentiment.get('aspect'))

            row = [tweet_text, ','.join(aspects)]
            writer.writerow(row)

# Ejemplo de uso:
xml_file = '../Data/socialtv-train-tagged.xml' # Reemplaza con la ruta a tu archivo XML
csv_file = 'tweets.csv' # Reemplaza con la ruta deseada para el archivo CSV
process_tweets(xml_file, csv_file)

print(f"Archivo CSV '{csv_file}' creado con éxito.")


Archivo CSV 'tweets.csv' creado con éxito.


In [ ]:
xml_file = '../Data/stompol-test-tagged.xml' # Reemplaza con la ruta a tu archivo XML

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

# 1. Carga del dataset
df = pd.read_csv("tu_dataset.csv") # Reemplaza "tu_dataset.csv" con el nombre de tu archivo

# 2. Preprocesamiento del dataset
# Convertir la columna 'aspecto' a una lista de etiquetas (necesario para BERT)
def etiqueta_aspectos(texto_aspectos):
    """Convierte la lista de aspectos en una lista de etiquetas para tokenización BERT."""
    aspectos = eval(texto_aspectos) # eval() es inseguro si tu CSV no es de confianza. Considera una función de parsing más robusta
    etiquetas = []
    for i, palabra in enumerate(texto.split()):
        if palabra in aspectos:
          etiquetas.append("B-ASPECTO") # B-ASPECTO indica el inicio de un aspecto
        else:
          etiquetas.append("O") # O indica que no es un aspecto

    return etiquetas


# Aplicar la función a cada fila
df['etiquetas'] = df.apply(lambda row: etiqueta_aspectos(row['aspecto']), axis=1)

# Limpieza adicional (opcional): elimina filas con etiquetas vacías o problemas de formato
df.dropna(subset=['etiquetas'], inplace=True)
df = df[df['etiquetas'].apply(lambda x: len(x) > 0)]


# 3. Crear un Dataset Hugging Face
dataset = Dataset.from_pandas(df)

# 4. Dividir el dataset en entrenamiento y validación
train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


# 5. Cargar el modelo BERT y el tokenizer
model_name = "bert-base-uncased" # Puedes usar otro modelo BERT pre-entrenado
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=2) # 2 etiquetas: O y B-ASPECTO


# 6. Tokenización del dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, is_split_into_words=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)


# 7. Alinear las etiquetas con los tokens
def align_labels_with_tokens(examples):
  labels = []
  for i, label in enumerate(examples["etiquetas"]):
      word_ids = examples["word_ids"][i]
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
          if word_idx is None:
              label_ids.append(-100)
          elif word_idx != previous_word_idx:
              label_ids.append(label[word_idx])
          else:
              label_ids.append(-100)
          previous_word_idx = word_idx
      labels.append(label_ids)
  examples["labels"] = labels
  return examples

tokenized_train_dataset = tokenized_train_dataset.map(align_labels_with_tokens, batched=True)
tokenized_eval_dataset = tokenized_eval_dataset.map(align_labels_with_tokens, batched=True)


# 8. Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3, # Ajusta el número de épocas según tus necesidades
    per_device_train_batch_size=8, # Ajusta el tamaño del batch
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Agrega otros argumentos de entrenamiento según sea necesario.
)


# 9. Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

trainer.train()

# 10. Guardar el modelo entrenado
trainer.save_model("./modelo_entrenado")